In [13]:
train_path='/Users/yezhuoyang/Desktop/ml/p2/cs420-2019-ctr-prediction/train.data'
test_path='/Users/yezhuoyang/Desktop/ml/p2/cs420-2019-ctr-prediction/test.data'
output_path='/Users/yezhuoyang/Desktop/ml/p2/cs420-2019-ctr-prediction/submission.csv'
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
def read_train(path):
    tag_list=[]
    data_list=[]
    with open(train_path,'r') as f:
           tmp=[]
           for line in f.readlines(): 
                tmp=line.strip().split(' ')
                if tmp[0]=='0':
                    tag_list.append(0)
                else:
                    tag_list.append(1)
                data_list.append(tmp[1:])
    return data_list,np.array(tag_list) 
def read_test(path):
    data_list=[]
    with open(path,'r') as f:
        for line in f.readlines():
            data_list.append(line.strip().split(' ')[1:])
    return data_list        
import scipy.sparse as sc
from sklearn.preprocessing import OneHotEncoder
def process_onehot(X):
    enc=OneHotEncoder(sparse=True)
    enc.fit(X)
    X=enc.transform(X)
    return X
def process_all(X_handout,X_origin):
     data=X_handout+X_origin
     enc=OneHotEncoder(sparse=True)
     enc.fit(data)
     data=enc.transform(data)
     return data[:len(X_handout)],data[len(X_handout):]
##稀疏矩阵onehot
X_origin,y=read_train(train_path)
X_handout=read_test(test_path)
X,X_handout=process_all(X_origin,X_handout)
X_train,X_test,y_train,y_test=train_test_split(X[:1000],y[:1000],test_size=.2, random_state=42)

In [23]:
import os,sys
print(os.getcwd())

/Users/yezhuoyang/ml


In [15]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

In [21]:
kfold = StratifiedKFold(n_splits=10)
X_train,X_test,y_train,y_test=train_test_split(X[:1000],y[:1000],test_size=.2, random_state=42)
from tqdm import tqdm

In [22]:
import pandas 
random_state = 2
classifiers = []
classifiers.append(SVC(random_state=random_state))
classifiers.append(DecisionTreeClassifier(random_state=random_state))
classifiers.append(AdaBoostClassifier(DecisionTreeClassifier(random_state=random_state),random_state=random_state,learning_rate=0.1))
classifiers.append(RandomForestClassifier(random_state=random_state))
classifiers.append(ExtraTreesClassifier(random_state=random_state))
classifiers.append(GradientBoostingClassifier(random_state=random_state))
classifiers.append(MLPClassifier(random_state=random_state))
classifiers.append(KNeighborsClassifier())
classifiers.append(LogisticRegression(random_state = random_state))
classifiers.append(LinearDiscriminantAnalysis())
cv_results = []
for classifier in tqdm(classifiers):
    s=cross_val_score(classifier,X_train,y_train,scoring="roc_auc", cv = kfold, n_jobs=2)
    print(s)
    cv_results.append(s)
cv_means = []
cv_std = []
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())
cv_res = pd.DataFrame({"CrossValMeans":cv_means,"CrossValerrors": cv_std,"Algorithm":["SVC","DecisionTree","AdaBoost",
"RandomForest","ExtraTrees","GradientBoosting","MultipleLayerPerceptron","KNeighboors","LogisticRegression","LinearDiscriminantAnalysis"]})
g = sns.barplot("CrossValMeans","Algorithm",data = cv_res, palette="Set3",orient = "h",**{'xerr':cv_std})
g.set_xlabel("Mean Accuracy")
g = g.set_title("Cross validation scores")

 10%|█         | 1/10 [00:03<00:29,  3.23s/it]

[0.37526652 0.39035591 0.44202067 0.57979334 0.54305396 0.5717566
 0.57634902 0.52238806 0.39265212 0.44871795]


 20%|██        | 2/10 [00:44<01:56, 14.60s/it]

[0.50586354 0.61653272 0.56314581 0.47129736 0.54707233 0.70091848
 0.47761194 0.53214696 0.54075775 0.61596737]


 30%|███       | 3/10 [01:29<02:45, 23.62s/it]

...

 50%|█████     | 5/10 [04:10<04:20, 52.06s/it]

[0.57249467 0.69804822 0.4793341  0.50688863 0.55166475 0.68771527
 0.53960964 0.58438576 0.52583238 0.62878788]


 60%|██████    | 6/10 [33:09<37:12, 558.02s/it]

[0.64712154 0.69517796 0.4695752  0.48564868 0.58438576 0.69919633
 0.53329506 0.68656716 0.6727899  0.63228438]


KeyboardInterrupt: 

In [243]:
import csv
def write(y_pred,path):
    i=0
    y_pred=list(y_pred)
    with open(path,'w',newline='') as f:
        csv_write=csv.writer(f)
        csv_write.writerow(['id','predicted'])
        for x in y_pred:
            csv_write.writerow([i,x])
            i=i+1
    f.close()   

In [244]:
write(y_pred,output_path)